<a href="https://colab.research.google.com/github/nilaygzl/TitanicML/blob/main/Titatnic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'titanic:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F3136%2F26502%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240716%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240716T131016Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Db7f30dc05be126b8036eba82a50df72c449d8f79fb30d0e191e8751ec9d305a613c1a671743b5f5daf0141c6814c8e9d5f0f1be585f30d5a81030654af7caf3bf5c9e2d46ce3f03176f2a46fc66e44f032bc63c267942cab0462c64eff03eaf8d386d26780c040c17161042cbce5d75f61c98bb00b764b059f347e3ad76c2db09c962812619c2de7613c586ae9c5ae4c784169b6adcad26d472d11a557d5bd495d3cd614e12ff97f63e34d2be0814e29b5b35df0feb20f824957d7429e702aa7b4601754a1cd0320a8661130ac0ff338f9cdf89786f638593ada1be78b7e481d86978a1f05e1c105d8b1e1bf40ad5b6010968a33c02901bc5e62aa098edd0c4b'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#kütüphaneleri yükle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#Veri dosyasını yükle
df_train=pd.read_csv('/kaggle/input/titanic/train.csv')
df_test=pd.read_csv('/kaggle/input/titanic/test.csv')

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
df_train.info()

In [ ]:
df_train.isnull().sum()

In [ ]:
df_train.describe().T

In [ ]:
df_train.describe()

In [ ]:
#veri ön işleme işlemleri yapılacağı için test ve train birleştirilir
df=pd.concat([df_train,df_test],ignore_index=True)
df

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
#satır sütun sayısı
df_train.shape,df_test.shape,df.shape #ilk 891 i train, son 418 i test

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
#numeric özellikler
numeric_columns=df.select_dtypes(include=['number'])
numeric_columns

In [ ]:
# Her çalışmada anlamlı çıkacak diye bir kural
numeric_columns.corr()

In [ ]:
df.isnull().sum()

In [ ]:
df['Embarked'].value_counts()

In [ ]:
df[df['Embarked'].isnull()]

In [ ]:
#Embarked ekiks veiryi doldur
#en çok tekrar eden değeri kullan
df['Embarked'].fillna(df['Embarked'].mode()[0],inplace=True) #data['Embarked'].mode()[0] birden fazla mod olabilir o yüzden ilkini al
df['Embarked'].isnull().sum()

In [ ]:
df['Age'].value_counts()

In [ ]:
df['Age'].isnull().sum()

In [ ]:
#Unvanları alalım
df['Title']=df['Name'].str.extract('([A-Za-z]+)\.')

In [ ]:
#hangi unvanlar var
df['Title'].value_counts()

In [ ]:
#Fransızca veya diğer dillerdeki ünvanları uygn şekilde çevirdik
df['Title']=df['Title'].replace(['Ms','Mlle'],'Miss')
df['Title']=df['Title'].replace(['Mme','Countess','Lady','Dona'],'Mrs')
df['Title']=df['Title'].replace(['Capt','Col','Don','Dr','Major','Rev','Sir','Jonkheer'],'Mr')

In [ ]:
df['Title'].value_counts()

In [ ]:
#Yaş değerlerini doldur
df['Age'].fillna(df.groupby('Title')['Age'].transform('mean'),inplace=True) # unvanların yaş ortalamalarına göre eksik yaşları doldur
df['Age'].isnull().sum()

In [ ]:
df.isnull().sum()

In [ ]:
df[df['Fare'].isnull()]

In [ ]:
df['Fare'].describe()

In [ ]:
df['Fare'].median()

In [ ]:
df['Fare'] = df.groupby(['Pclass', 'Embarked'])['Fare'].transform(lambda x: x.fillna(x.median()))
df['Fare'].isnull().sum()

In [ ]:
df.isnull().sum()

In [ ]:
df = df.drop(['Cabin'],axis=1)

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
# "Pclass" sütununu kategorik veri türüne dönüştürme
df['Pclass'] = df['Pclass'].astype('category')

# Dönüşümün başarılı olup olmadığını kontrol etme
print(df['Pclass'].dtype)

In [ ]:
df.info()

In [ ]:
df['Sex'].value_counts()

In [ ]:
df['Pclass'].value_counts()

In [ ]:
# Pclass sütununu One-Hot encoding yapma, 2 sütunla temsil etme
pclass_one_hot = pd.get_dummies(df['Pclass'], prefix='Pclass', drop_first=True)

# Oluşan One-Hot encoding verilerini orijinal veri setine eklemek
df = pd.concat([df, pclass_one_hot], axis=1)

In [ ]:
# Sex sütununu One-Hot encoding yapma, sadece 'male' olanlar için 1 yapma
sex_one_hot = pd.get_dummies(df['Sex'], prefix='Sex', drop_first=True)

# Oluşan One-Hot encoding verilerini orijinal veri setine eklemek
df = pd.concat([df, sex_one_hot], axis=1)

In [ ]:
# Embarked sütununu One-Hot encoding yapma, 2 sütunla temsil etme
Embarked_one_hot = pd.get_dummies(df['Embarked'], prefix='Embarked', drop_first=True)

# Oluşan One-Hot encoding verilerini orijinal veri setine eklemek
df = pd.concat([df, Embarked_one_hot], axis=1)

In [ ]:
# Title sütununu One-Hot encoding yapma, 2 sütunla temsil etme
Title_one_hot = pd.get_dummies(df['Title'], prefix='Title')

# Oluşan One-Hot encoding verilerini orijinal veri setine eklemek
df = pd.concat([df, Title_one_hot], axis=1)

In [ ]:
df.info()

In [ ]:
data_encoded = df.copy()

In [ ]:
columns_to_drop = ['PassengerId', 'Pclass', 'Name', 'Sex', 'Ticket', 'Embarked', 'Title']
data_encoded.drop(columns=columns_to_drop, inplace=True, errors='ignore')

In [ ]:
data_encoded.info()

In [ ]:
data_encoded = data_encoded.loc[:, ~data_encoded.columns.duplicated()]
data_encoded.info()

In [ ]:
# Korelasyon matrisini oluşturma
correlation_matrix = data_encoded.corr()

# Heatmap çizdirme
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", annot_kws={"size": 10})
plt.title('Correlation Heatmap of Titanic Dataset')
plt.show()

In [ ]:
# Fare, Parch, SibSp, Age sütunlarından polinomlar oluşturma
for feature in ['Fare', 'Age']:
    for degree in [2, 3]:  # Polinom derecesi 2 ve 3 olarak örnek verilmiştir
        new_column_name = f'{feature}_poly_{degree}'
        data_encoded[new_column_name] = np.power(data_encoded[feature], degree)

# İlişkisel sütunlar oluşturma (örnek olarak Fare ve Age arasındaki etkileşim)
data_encoded['Fare_Age_interaction'] = data_encoded['Fare'] * data_encoded['Age']
data_encoded['SibSp_Parch_interaction'] = data_encoded['SibSp'] * data_encoded['Parch']

In [ ]:
y = data_encoded['Survived']
X = data_encoded.drop(columns=['Survived'])  # Survived dışındaki tüm sütunlar
X.head()

In [ ]:
y

In [ ]:
#Train ve testi aşağıdaki gibi ayırıyoruz
X_train=X[:891] #Train veri setinde ilk 891
X_test=X[891:]
y_train=y[:891]
y_test=y[891:]

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor


# RandomForestRegressor modelini oluşturma ve eğitme
rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)

# DecisionTreeRegressor modelini oluşturma ve eğitme
dt_model = DecisionTreeRegressor()
dt_model.fit(X_train, y_train)

# GradientBoostingRegressor modelini oluşturma ve eğitme
gb_model = GradientBoostingRegressor()
gb_model.fit(X_train, y_train)

# Özellik önemlerini belirleme
rf_feature_importances = pd.Series(rf_model.feature_importances_, index=X.columns).sort_values(ascending=True)
dt_feature_importances = pd.Series(dt_model.feature_importances_, index=X.columns).sort_values(ascending=True)
gb_feature_importances = pd.Series(gb_model.feature_importances_, index=X.columns).sort_values(ascending=True)

# Grafik yüksekliğini ayarlama
num_features = len(rf_feature_importances)
fig_height = max(6, num_features * 0.5)

# Grafik oluşturma
plt.figure(figsize=(12, fig_height))

# RandomForestRegressor özellik önemleri
plt.subplot(3, 1, 1)
rf_feature_importances.plot(kind='barh')
plt.title('Random Forest Özellik Önem Düzeyleri')
plt.xlabel('Önem Düzeyi')
plt.ylabel('Özellikler')

# DecisionTreeRegressor özellik önemleri
plt.subplot(3, 1, 2)
dt_feature_importances.plot(kind='barh')
plt.title('Decision Tree Özellik Önem Düzeyleri')
plt.xlabel('Önem Düzeyi')
plt.ylabel('Özellikler')

# GradientBoostingRegressor özellik önemleri
plt.subplot(3, 1, 3)
gb_feature_importances.plot(kind='barh')
plt.title('Gradient Boosting Özellik Önem Düzeyleri')
plt.xlabel('Önem Düzeyi')
plt.ylabel('Özellikler')

# Barların üstüne değerlerini yazma
for ax in plt.gcf().axes:
    for container in ax.containers:
        ax.bar_label(container, label_type='edge')

plt.tight_layout()


# Excel'e yazdırma
feature_importances_df = pd.DataFrame({
    'features': X.columns,
    'rf_importance': rf_feature_importances[X.columns].values,
    'dt_importance': dt_feature_importances[X.columns].values,
    'gb_importance': gb_feature_importances[X.columns].values
})



plt.show()

print(feature_importances_df)

libraries

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error,  accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.neural_network import MLPClassifier
import warnings

from sklearn.exceptions import ConvergenceWarning

In [ ]:
# Ignore ConvergenceWarnings
warnings.filterwarnings(action='ignore', category=ConvergenceWarning)


X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.3, random_state=42)




# Model listesi
models = [
    ('LogisticRegression', LogisticRegression()),
    ('DecisionTree', DecisionTreeClassifier()),
    ('RandomForest', RandomForestClassifier()),
    ('GradientBoosting', GradientBoostingClassifier()),
    ('AdaBoost', AdaBoostClassifier()),
    ('ExtraTrees', ExtraTreesClassifier()),
    ('Bagging', BaggingClassifier()),
    ('KNeighbors', KNeighborsClassifier()),
    ('QDA', QuadraticDiscriminantAnalysis()),
    ('LDA', LinearDiscriminantAnalysis()),
    ('XGB', XGBClassifier(use_label_encoder=False, eval_metric='logloss')),
    ('LGBM', LGBMClassifier(verbosity=-1)),
    ('CatBoost', CatBoostClassifier(silent=True)),
    ('GaussianNB', GaussianNB()),
    ('BernoulliNB', BernoulliNB()),
    ('MultinomialNB', MultinomialNB()),
    ('MLP', MLPClassifier(max_iter=300))
]

# Hiperparametreler için grid
param_grids = {
    'LogisticRegression': {'C': [0.1, 1, 10]},
    'DecisionTree': {'max_depth': [3, 5, 7]},
    'RandomForest': {'n_estimators': [50, 100, 200], 'max_depth': [3, 5, 7]},
    'GradientBoosting': {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 0.2]},
    'AdaBoost': {'n_estimators': [50, 100, 200]},
    'ExtraTrees': {'n_estimators': [50, 100, 200], 'max_depth': [3, 5, 7]},
    'Bagging': {'n_estimators': [50, 100, 200]},
    'KNeighbors': {'n_neighbors': [3, 5, 7]},
    'QDA': {},
    'LDA': {},
    'XGB': {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 0.2]},
    'LGBM': {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 0.2]},
    'CatBoost': {'iterations': [50, 100, 200], 'learning_rate': [0.01, 0.1, 0.2]},
    'GaussianNB': {},
    'BernoulliNB': {'alpha': [0.01, 0.1, 1]},
    'MultinomialNB': {'alpha': [0.01, 0.1, 1]},
    'MLP': {'hidden_layer_sizes': [(100,), (100, 50), (50, 50)], 'activation': ['relu', 'tanh']}
}

# Modellerin eğitimi ve değerlendirilmesi
results = []
for name, model in models:
    print(f"Training {name}")
    param_grid = param_grids.get(name, {})
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    cm = confusion_matrix(y_test, y_pred)

    results.append({
        'Model': name,
        'Best Params': grid_search.best_params_,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Confusion Matrix': cm
    })

# Sonuçları yazdırma
for result in results:
    print(f"Model: {result['Model']}")
    print(f"Best Params: {result['Best Params']}")
    print(f"Accuracy: {result['Accuracy']}")
    print(f"Precision: {result['Precision']}")
    print(f"Recall: {result['Recall']}")
    print(f"F1 Score: {result['F1 Score']}")
    print(f"Confusion Matrix:\n{result['Confusion Matrix']}")
    print("\n")

In [ ]:
#Train ve testi aşağıdaki gibi ayırıyoruz
X_train=X[:891] #Train veri setinde ilk 891
X_test=X[891:]
y_train=y[:891]
y_test=y[891:]

In [ ]:
XTC=ExtraTreesClassifier(max_depth=5, n_estimators=200)
XTC.fit(X_train,y_train)# Testi kaggle üzerinden yapacağımız için artık veri setini bölmeden elimizdeki tüm veri setini gönderiyoruz.
y_pred=XTC.predict(X_test)
y_pred

In [ ]:
submission = pd.DataFrame({'PassengerId': df_test.PassengerId, 'Survived': y_pred.astype(int)})
submission

In [ ]:
submission.to_csv('submission.csv', index=False) #data frame i csv dosyası oalrak kaydediyor